In [21]:
import os
import rasterio
from rasterio.merge import merge
from shapely.geometry import mapping, Polygon, Point
from rasterio import features
import fiona
from fiona.crs import from_epsg
from osgeo import gdal
import ogr, osr
from osgeo import ogr
import csv
import geojson
import json
import pandas
from rasterstats import raster_stats
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score,confusion_matrix, cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import tsraster.model  as md
import pandas as pd
from rasterio.warp import calculate_default_transform, reproject, Resampling
from tsraster.calculate import calculateFeatures
import seaborn as sb
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'rasterio'

## Download


**conda activate planet_down environment*
*from shell cd to location of get_planet_v2**

running for dates:

"gte":"2016-10-15T01:00:00.000Z", "lte":"2016-10-21T23:00:00.000Z"

"gte":"2016-11-15T01:00:00.000Z", "lte":"2016-11-21T23:00:00.000Z"

"gte":"2016-12-01T01:00:00.000Z", "lte":"2016-12-15T23:00:00.000Z"

python get_planet_v2.py --path C:\Users\mmann\Documents\GitHub\IFPRI_Fire_India\Images_new --filter ./example.json --key YOUR_API_KEY

# Merge 

*create composites of images for same day*

First see if we can get gdal working

In [ ]:
!gdalinfo "./Images_new/4352923_2016-12-01_RE2_3A_Analytic_clip.tif"

Get unique dates to merge

In [1]:
import glob 

octo = set(glob.glob('./Images_new/*2016*10*1*AnalyticMS*.tif'))-set(glob.glob('./Images_new/*DN*'))
octo = ' '.join(str(elem) for elem in octo)
print(octo)

nov = set(glob.glob('./Images_new/*2016*11*AnalyticMS*.tif'))-set(glob.glob('./Images_new/*DN*'))
nov = ' '.join(str(elem) for elem in nov)
print(nov)

# get 12-01
dec = set(glob.glob('./Images_new/201612*AnalyticMS*.tif'))-set(glob.glob('./Images_new/*DN*'))
dec = ' '.join(str(elem) for elem in dec)
print(dec)

 



./Images_new\20161018_044250_0e1f_3B_AnalyticMS_clip.tif ./Images_new\20161017_055724_0c19_3B_AnalyticMS_clip.tif ./Images_new\20161017_044241_0e0d_3B_AnalyticMS_clip.tif ./Images_new\20161017_055726_0c19_3B_AnalyticMS_clip.tif ./Images_new\20161017_044242_0e0d_3B_AnalyticMS_clip.tif ./Images_new\20161205_101811_0c75_3B_AnalyticMS_clip.tif ./Images_new\20161017_055723_0c19_3B_AnalyticMS_clip.tif ./Images_new\20161017_044240_0e0d_3B_AnalyticMS_clip.tif ./Images_new\20161017_044243_0e0d_3B_AnalyticMS_clip.tif ./Images_new\20161018_044252_0e1f_3B_AnalyticMS_clip.tif ./Images_new\20161018_044251_0e1f_3B_AnalyticMS_clip.tif ./Images_new\20161017_055725_0c19_3B_AnalyticMS_clip.tif
./Images_new\20161124_044355_0e19_3B_AnalyticMS.tif ./Images_new\20161121_044424_0e3a_3B_AnalyticMS.tif ./Images_new\20161118_044452_0e20_3B_AnalyticMS_clip.tif ./Images_new\20161118_044450_0e20_3B_AnalyticMS_clip.tif ./Images_new\20161116_044440_0e3a_3B_AnalyticMS_clip.tif ./Images_new\20161121_044425_0e3a_3B_Anal

In [19]:
# create gdal_calc expressions

from string import ascii_uppercase

octo = set(glob.glob('./Images_new/*2016*10*1*AnalyticMS*.tif'))-set(glob.glob('./Images_new/*DN*'))
octo_files = ' '.join('-'+letter +' '+ str(image) for image, letter in zip(octo, ascii_uppercase))
print(octo_files)
octo_exp = '('+' + '.join( letter   for image, letter in zip(octo, ascii_uppercase)) + ')/' +str(len(octo))
print(octo_exp)

nov = set(glob.glob('./Images_new/*2016*11*AnalyticMS*.tif'))-set(glob.glob('./Images_new/*DN*'))
nov_files = ' '.join('-'+letter +' '+ str(image) for image, letter in zip(nov, ascii_uppercase))
print(nov_files)
nov_exp = '('+' + '.join( letter   for image, letter in zip(nov, ascii_uppercase)) + ')/' +str(len(octo))
print(nov_exp)

dec = set(glob.glob('./Images_new/201612*AnalyticMS*.tif'))-set(glob.glob('./Images_new/*DN*'))
dec_files = ' '.join('-'+letter +' '+ str(image) for image, letter in zip(dec, ascii_uppercase))
print(dec_files)

dec_exp = '('+' + '.join( letter   for image, letter in zip(dec, ascii_uppercase)) + ')/' +str(len(dec))
dec_exp





-A ./Images_new\20161018_044250_0e1f_3B_AnalyticMS_clip.tif -B ./Images_new\20161017_055724_0c19_3B_AnalyticMS_clip.tif -C ./Images_new\20161017_044241_0e0d_3B_AnalyticMS_clip.tif -D ./Images_new\20161017_055726_0c19_3B_AnalyticMS_clip.tif -E ./Images_new\20161017_044242_0e0d_3B_AnalyticMS_clip.tif -F ./Images_new\20161205_101811_0c75_3B_AnalyticMS_clip.tif -G ./Images_new\20161017_055723_0c19_3B_AnalyticMS_clip.tif -H ./Images_new\20161017_044240_0e0d_3B_AnalyticMS_clip.tif -I ./Images_new\20161017_044243_0e0d_3B_AnalyticMS_clip.tif -J ./Images_new\20161018_044252_0e1f_3B_AnalyticMS_clip.tif -K ./Images_new\20161018_044251_0e1f_3B_AnalyticMS_clip.tif -L ./Images_new\20161017_055725_0c19_3B_AnalyticMS_clip.tif
(A + B + C + D + E + F + G + H + I + J + K + L)/12
-A ./Images_new\20161124_044355_0e19_3B_AnalyticMS.tif -B ./Images_new\20161121_044424_0e3a_3B_AnalyticMS.tif -C ./Images_new\20161118_044452_0e20_3B_AnalyticMS_clip.tif -D ./Images_new\20161118_044450_0e20_3B_AnalyticMS_clip.tif

'(A + B + C + D + E + F + G + H + I + J + K + L + M + N)/14'

Paste into command line:

In [ ]:
python C:\Users\mmann\AppData\Local\Continuum\anaconda3\envs\ts-raster\Lib\site-packages\osgeo\scripts\gdal_merge.py -of gtiff  -o October_mosaic.tif -v ./Images_new\20161017_044243_0e0d_3B_AnalyticMS_clip.tif ./Images_new\20161017_055723_0c19_3B_AnalyticMS_clip.tif ./Images_new\20161017_055725_0c19_3B_AnalyticMS_clip.tif ./Images_new\20161205_101811_0c75_3B_AnalyticMS_clip.tif ./Images_new\20161018_044251_0e1f_3B_AnalyticMS_clip.tif ./Images_new\20161017_044242_0e0d_3B_AnalyticMS_clip.tif ./Images_new\20161018_044250_0e1f_3B_AnalyticMS_clip.tif ./Images_new\20161017_044240_0e0d_3B_AnalyticMS_clip.tif ./Images_new\20161018_044252_0e1f_3B_AnalyticMS_clip.tif ./Images_new\20161017_055724_0c19_3B_AnalyticMS_clip.tif ./Images_new\20161017_044241_0e0d_3B_AnalyticMS_clip.tif ./Images_new\20161017_055726_0c19_3B_AnalyticMS_clip.tif

In [ ]:
python C:\Users\mmann\AppData\Local\Continuum\anaconda3\envs\ts-raster\Lib\site-packages\osgeo\scripts\gdal_merge.py -of gtiff -a_nodata -9999 -pct  -o November_mosaic.tif -v ./Images_new\20161205_101811_0c75_3B_AnalyticMS_clip.tif ./Images_new\20161121_044424_0e3a_3B_AnalyticMS.tif ./Images_new\20161124_044357_0e19_3B_AnalyticMS.tif ./Images_new\20161116_044315_0e0d_3B_AnalyticMS_clip.tif ./Images_new\20161118_044450_0e20_3B_AnalyticMS_clip.tif ./Images_new\20161116_044313_0e0d_3B_AnalyticMS_clip.tif ./Images_new\20161124_044358_0e19_3B_AnalyticMS.tif ./Images_new\20161121_044425_0e3a_3B_AnalyticMS.tif ./Images_new\20161116_044441_0e3a_3B_AnalyticMS_clip.tif ./Images_new\20161124_044355_0e19_3B_AnalyticMS.tif ./Images_new\20161118_044452_0e20_3B_AnalyticMS_clip.tif ./Images_new\20161116_044316_0e0d_3B_AnalyticMS_clip.tif ./Images_new\20161121_044426_0e3a_3B_AnalyticMS.tif ./Images_new\20161116_044439_0e3a_3B_AnalyticMS_clip.tif ./Images_new\20161116_044440_0e3a_3B_AnalyticMS_clip.tif ./Images_new\20161124_044356_0e19_3B_AnalyticMS.tif

In [20]:
python C:\Users\mmann\AppData\Local\Continuum\anaconda3\envs\ts-raster\Lib\site-packages\osgeo\scripts\gdal_merge.py  -a_nodata -9999 -pct -of gtiff  -o December_mosaic.tif -v ./Images_new\20161219_043945_0c42_3B_AnalyticMS.tif ./Images_new\20161213_070355_0c24_3B_AnalyticMS.tif ./Images_new\20161219_043942_0c42_3B_AnalyticMS.tif ./Images_new\20161205_101811_0c75_3B_AnalyticMS_clip.tif ./Images_new\20161219_043943_0c42_3B_AnalyticMS.tif ./Images_new\20161219_044125_0c41_3B_AnalyticMS.tif ./Images_new\20161201_044316_0e3a_3B_AnalyticMS_clip.tif ./Images_new\20161213_070354_0c24_3B_AnalyticMS.tif ./Images_new\20161213_070353_0c24_3B_AnalyticMS.tif ./Images_new\20161213_070355_1_0c24_3B_AnalyticMS.tif ./Images_new\20161219_044127_1_0c41_3B_AnalyticMS.tif ./Images_new\20161201_044317_0e3a_3B_AnalyticMS_clip.tif ./Images_new\20161219_043944_0c42_3B_AnalyticMS.tif ./Images_new\20161219_044126_0c41_3B_AnalyticMS.tif

SyntaxError: invalid syntax (<ipython-input-20-112efce34d9a>, line 1)

In [ ]:
python C:\Users\mmann\AppData\Local\Continuum\anaconda2\envs\ts-rasterv3\Lib\site-packages\osgeo\scripts\gdal_calc.py -A ./Images_new\20161018_044250_0e1f_3B_AnalyticMS_clip.tif -B ./Images_new\20161017_055724_0c19_3B_AnalyticMS_clip.tif -C ./Images_new\20161017_044241_0e0d_3B_AnalyticMS_clip.tif -D ./Images_new\20161017_055726_0c19_3B_AnalyticMS_clip.tif -E ./Images_new\20161017_044242_0e0d_3B_AnalyticMS_clip.tif -F ./Images_new\20161205_101811_0c75_3B_AnalyticMS_clip.tif -G ./Images_new\20161017_055723_0c19_3B_AnalyticMS_clip.tif -H ./Images_new\20161017_044240_0e0d_3B_AnalyticMS_clip.tif -I ./Images_new\20161017_044243_0e0d_3B_AnalyticMS_clip.tif -J ./Images_new\20161018_044252_0e1f_3B_AnalyticMS_clip.tif -K ./Images_new\20161018_044251_0e1f_3B_AnalyticMS_clip.tif -L ./Images_new\20161017_055725_0c19_3B_AnalyticMS_clip.tif --outfile=October_mosaic.tif --calc="(A + B + C + D + E + F + G + H + I + J + K + L)/12" --NoDataValue=0

# Calculate Features


In [ ]:
# Function to calculate the Burn Area Index for each raster
def bai_rast(inpath, outpath):
    
    # Opens the raster for the calculation
    with rasterio.open(inpath) as src:
        brn_data = src.read()
       
    # BAI Formula
    brn_bai = (1 / ((.1-brn_data[2])**2)+((.06-brn_data[3])**2))
    
    # Sets up the metadata for the output raster
    with rasterio.open(inpath) as src:
        brn_data_ras = src.read()
        brn_meta = src.profile

    brn_transform = brn_meta["transform"]
    brn_crs = brn_meta["crs"]
    brn_meta['count'] = 1
    brn_meta['dtype'] = 'float64'

    # Saves out the raster according to the below criteria
    with rasterio.open(outpath, 'w', **brn_meta) as dst:
        dst.write(brn_bai, 1)

# Function to select files to calculate the BAI on
def compare2():
    ending_list = []
    for root, dirs, files in os.walk(inpath):
        for filename in files:
            if filename.endswith("merge.tif"):
                print('File name is: {0}'.format(filename))
                ending_list.append(filename.split('_',1)[0])

    unique_ending = list(set(ending_list))
    unique2 = unique_ending
    
    # Selects the files using a unique identifier and sets up the out paths
    for unique in unique_ending:
        for root, dirs, files in os.walk(inpath):
            for filename in files:
                if filename.startswith(unique):
                    files_unique = [x for x in files if x.startswith(unique)]
                    file_paths = [x for x in files_unique if x.endswith("merge.tif")]
                    outpath = inpath + unique + "_bai.tif"
                    if unique != False:
                        os.chdir(inpath)
                        bai_rast(file_paths[0], outpath)
               

compare2()